In [2]:
import pandas as pd
import os

data = pd.read_csv("../data/raw/WineQT.csv")

In [3]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64

In [4]:
def missing_values(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
    
missing_values_table = missing_values(data)
missing_values_table

Your selected dataframe has 13 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [ ]:
import missingno as msno # For visualizing missing data patterns

# To see how the data correlate with each other;

sorted_data = data.sort_values('oldpeak')
msno.matrix(sorted_data)

In [ ]:
msno.heatmap(data)

In [ ]:
msno.dendrogram(data)

In [ ]:
from sklearn.preprocessing import LabelEncoder # For converting categorical data to numerical values
from sklearn.model_selection import train_test_split # For splitting data into training and testing sets
from sklearn.metrics import accuracy_score, f1_score # For calculating accuracy and F1-score
from sklearn.impute import KNNImputer # For imputing missing values using K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier # K-Nearest Neighbors classification model
from sklearn.linear_model import LinearRegression, LogisticRegression # Linear regression and logistic regression models
from sklearn.preprocessing import StandardScaler # For scaling features
from sklearn.ensemble import RandomForestRegressor # Random Forest regression model
from sklearn.ensemble import RandomForestClassifier # Random Forest classifier model
from sklearn.metrics import mean_squared_error # Performance metric for regression models
from sklearn.impute import SimpleImputer # For imputing missing values with a simple strategy
from sklearn.experimental import enable_iterative_imputer # Enable IterativeImputer (needed for MICE/MissForest)
from sklearn.impute import IterativeImputer # IterativeImputer for MICE and MissForest

from xgboost import XGBClassifier # XGBoost classification model
from xgboost import XGBRegressor # XGBoost regression model 
import numpy as np


# Import imputation techniques from imputation_techniques.py
from imputation_techniques import impute_knn, impute_missForest, impute_mice, pool_mice_results

In [ ]:
def read_data():
    """
    Reads the 'Heart disease' dataset,
    performs some editing, and returns it.
    """
    df = pd.read_csv("../data/raw/heart_disease_uci.csv")
    #df['num'] = df['num'].replace({2: 1, 3: 1, 4: 1})
    df = df.drop('id', axis=1)
    df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
    
    categorical_columns = df.select_dtypes(include=['category']).columns

    # Convert categorical to numerical
    label_encoder = LabelEncoder()
    
    for column in categorical_columns:
        df[column] = label_encoder.fit_transform(df[column])
    return df

def split_dataset(df):
    """
    Splits the dataset into training and testing sets.
    
    Parameters:
    - df (pd.DataFrame): The input dataset.
    
    Returns:
    - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
    """
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # print(f'Size of the Train: {X_train.shape[0]}')
    # print(f'Size of the Test: {X_test.shape[0]}')
    return X_train, X_test, y_train, y_test

def train_model_XGBoost(X_train, X_test, y_train, y_test):
    """
    Trains an XGBoost model on the given training set and evaluates it on the test set.
    
    Parameters:
    - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
    """
    # Create and train the XGBoost model
    xgb_model = XGBClassifier()
    xgb_model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = xgb_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}\n")
    return accuracy, f1
    
def train_model_random_forest(X_train, X_test, y_train, y_test):
    """
    Creates and trains a RandomForest model on the given training set 
    and evaluates its performance on the test set.
    
    Parameters:
    - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
    
    Returns:
    - accuracy, f1_score (float): Accuracy and F1-Score of the trained model on the test set.
    """
    # Create and train the RandomForest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}\n")
    return accuracy, f1

def train_model_logistic(X_train, X_test, y_train, y_test):
    """
    Creates and trains a LogisticRegression model on the given training set 
    and evaluates its performance on the test set.
    
    Parameters:
    - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
    
    Returns:
    - accuracy, f1_score (float): Accuracy and F1-Score of the trained model on the test set.
    """
    # Scale features for LogisticRegression convergence
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create and train the LogisticRegression model
    lr_model = LogisticRegression(max_iter=50000)
    lr_model.fit(X_train_scaled, y_train)

    # Evaluate the model on the test set
    y_pred = lr_model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}\n")
    return accuracy, f1

# To compare the success of the methods
accuracies = {}
f1_scores = {}

# READ AND SPLIT DATA ONCE - ALL METHODS WILL USE THIS SAME SPLIT

print("Loading and splitting data once for all methods...")

df_original = read_data()
X_train_original, X_test_original, y_train_original, y_test_original = split_dataset(df_original)
print(f"Train size: {X_train_original.shape[0]}, Test size: {X_test_original.shape[0]}\n")

In [ ]:

# Check class balance
print("\nClass Distribution (Target Variable 'num'):")
print(y_train_original.value_counts(normalize=True))
print("\nNote: Imbalanced classes can lead to lower accuracy!")
print("="*60)


In [ ]:
# METHOD 1: No Handling (use original split with missing values)
print("=" * 50)
print("METHOD 1: No Handling")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Train an XGBoost model on the dataset without handling missing values
accuracy_no_handling, f1_no_handling = train_model_XGBoost(X_train, X_test, y_train, y_test)

# Store the accuracy and F1-score in dictionaries
accuracies['No Handling'] = accuracy_no_handling
f1_scores['No Handling'] = f1_no_handling

# Show a specific row example (no imputation done in this method)
#print("\n------------------\n")
#print("\nExample row with missing values: \n")
#print(X_train[(X_train['age']==58.0) & (X_train['sex']==1.0) & (X_train['dataset']==3.0) & (X_train['chol']==385)])


In [ ]:
# METHOD 2: Pairwise Deletions (drop rows with ANY missing values)
print("=" * 50)
print("METHOD 2: Pairwise Deletions")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Drop rows with any missing values
train_mask = X_train.notna().all(axis=1)
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = X_test.notna().all(axis=1)
X_test = X_test[test_mask]
y_test = y_test[test_mask]

print(f"After deletion - Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

# Train both Random Forest and Logistic Regression models
print("\nXGBoost:")
accuracy_pairwise_xgb, f1_pairwise_xgb = train_model_XGBoost(X_train, X_test, y_train, y_test)
print("\nRandom Forest:")
accuracy_pairwise_rf, f1_pairwise_rf = train_model_random_forest(X_train, X_test, y_train, y_test)
print("\nLogistic Regression:")
accuracy_pairwise_lr, f1_pairwise_lr = train_model_logistic(X_train, X_test, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['Pairwise_XGB'] = accuracy_pairwise_xgb
accuracies['Pairwise_RF'] = accuracy_pairwise_rf
accuracies['Pairwise_LR'] = accuracy_pairwise_lr

f1_scores['Pairwise_XGB'] = f1_pairwise_xgb
f1_scores['Pairwise_LR'] = f1_pairwise_lr
f1_scores['Pairwise_RF'] = f1_pairwise_rf

In [ ]:
# METHOD 3: Listwise Deletions (drop rows with missing values in specific columns)
print("=" * 50)
print("METHOD 3: Listwise Deletions")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Drop rows with missing values in specific columns
cols_to_check = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
train_mask = X_train[cols_to_check].notna().all(axis=1)
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = X_test[cols_to_check].notna().all(axis=1)
X_test = X_test[test_mask]
y_test = y_test[test_mask]

print(f"After deletion - Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

# Train both Random Forest and Logistic Regression models
print("\nXGBoost:")
accuracy_listwise_xgb, f1_listwise_xgb = train_model_XGBoost(X_train, X_test, y_train, y_test)
print("\nRandom Forest:")
accuracy_listwise_rf, f1_listwise_rf = train_model_random_forest(X_train, X_test, y_train, y_test)
print("\nLogistic Regression:")
accuracy_listwise_lr, f1_listwise_lr = train_model_logistic(X_train, X_test, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['Listwise_XGB'] = accuracy_listwise_xgb
accuracies['Listwise_RF'] = accuracy_listwise_rf
accuracies['Listwise_LR'] = accuracy_listwise_lr

f1_scores['Listwise_XGB'] = f1_listwise_xgb
f1_scores['Listwise_LR'] = f1_listwise_lr
f1_scores['Listwise_RF'] = f1_listwise_rf

In [ ]:
# METHOD 4: Dropping Entire Columns (drop columns with ANY missing values)
print("=" * 50)
print("METHOD 4: Dropping Entire Columns")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Drop entire columns with any missing values
cols_with_missing = X_train.columns[X_train.isna().any()].tolist()
print(f"Dropping columns: {cols_with_missing}")
X_train = X_train.drop(columns=cols_with_missing)
X_test = X_test.drop(columns=cols_with_missing)

print(f"Remaining features: {X_train.shape[1]}")

# Train both Random Forest and Logistic Regression models
print("\nRandom Forest:")
accuracy_dropping_rf, f1_dropping_rf = train_model_random_forest(X_train, X_test, y_train, y_test)
print("\nLogistic Regression:")
accuracy_dropping_lr, f1_dropping_lr = train_model_logistic(X_train, X_test, y_train, y_test)
print("\nXGBoost:")
accuracy_dropping_xgb, f1_dropping_xgb = train_model_XGBoost(X_train, X_test, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['Dropping_Columns_RF'] = accuracy_dropping_rf
accuracies['Dropping_Columns_LR'] = accuracy_dropping_lr
accuracies['Dropping_Columns_XGB'] = accuracy_dropping_xgb
f1_scores['Dropping_Columns_RF'] = f1_dropping_rf
f1_scores['Dropping_Columns_LR'] = f1_dropping_lr
f1_scores['Dropping_Columns_XGB'] = f1_dropping_xgb

# A specific data, after the procedure

#print("\n------------------\n")#X_train[(X_train['age']==58.0) & (X_train['sex']==1.0) & (X_train['dataset']==3.0) & (X_train['restecg']==0) & (X_train['exang']==2)]
#print("\nAfter Imputation: \n")

In [ ]:
def impute_and_train(X_train, y_train, X_test, y_test, strategy):
    """
    Imputes missing values in the training and testing sets using the specified strategy,
    trains both Random Forest and Logistic Regression models, and evaluates their performance.
    
    Parameters:
    - X_train, y_train: Training features and labels.
    - X_test, y_test: Testing features and labels.
    - strategy (str): Imputation strategy to handle missing values ('mean' or 'median').
    
    Returns:
    - accuracy_rf, accuracy_lr (float): Accuracy scores of both models on the test set.
    - X_train_imputed: Imputed training data
    """
    # Impute missing values
    imputer = SimpleImputer(strategy=strategy)
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    
    # Train both models on the imputed training set
    print(f"\n\nImputation strategy: {strategy}")
    
    print("Random Forest:")
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_imputed, y_train)
    y_pred_rf = rf_model.predict(X_test_imputed)
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy_rf:.4f}, F1-Score: {f1_rf:.4f}")
    
    print("\nLogistic Regression:")
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train_imputed, y_train)
    y_pred_lr = lr_model.predict(X_test_imputed)
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    f1_lr = f1_score(y_test, y_pred_lr, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy_lr:.4f}, F1-Score: {f1_lr:.4f}\n")

    print("\nXGBOOST:")
    xgb_model = XGBClassifier()
    xgb_model.fit(X_train_imputed, y_train)
    y_pred_xgb = xgb_model.predict(X_test_imputed)
    accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
    f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
    print(f"Accuracy: {accuracy_xgb:.4f}, F1-Score: {f1_xgb:.4f}\n")
    
    return accuracy_rf, accuracy_lr, accuracy_xgb, f1_rf, f1_lr, f1_xgb, X_train_imputed

# Use the same original split
X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Train the model using different imputation strategies
strategies = ['mean', 'median']

for strategy in strategies:
    print("=" * 50)
    print(f"METHOD: {strategy.upper()} Imputation")
    print("=" * 50)
    
    accuracy_rf, accuracy_lr, accuracy_xgb, f1_rf, f1_lr, f1_xgb, X_train_imputed = impute_and_train(X_train, y_train, X_test, y_test, strategy)
    accuracies[f'{strategy}_RF'] = accuracy_rf
    accuracies[f'{strategy}_LR'] = accuracy_lr
    accuracies[f'{strategy}_XGB'] = accuracy_xgb
    f1_scores[f'{strategy}_RF'] = f1_rf
    f1_scores[f'{strategy}_LR'] = f1_lr
    f1_scores[f'{strategy}_XGB'] = f1_xgb
    
    # A specific data, after the procedure
    #print("\n------------------\n")
    #print("\nAfter Imputation: \n")

    #print(X_train_imputed[(X_train_imputed['age']==58.0) & (X_train_imputed['sex']==1.0) & (X_train_imputed['dataset']==3.0) & (X_train_imputed['chol']==385)])    print("\n----------------------------\n----------------------------\n----------------------------")

In [ ]:
# METHOD 5: KNN Imputation (k=5)
print("=" * 50)
print("METHOD: KNN Imputation (k=5)")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Create combined DataFrame for imputation (KNN needs all data together)
X_train_knn = impute_knn(X_train, n_neighbors=5)
X_test_knn = impute_knn(X_test, n_neighbors=5)

print("\nXGBoost:")
accuracy_knn5_xgb, f1_knn5_xgb = train_model_XGBoost(X_train_knn, X_test_knn, y_train, y_test)
print("\nRandom Forest:")
accuracy_knn5_rf, f1_knn5_rf = train_model_random_forest(X_train_knn, X_test_knn, y_train, y_test)
print("\nLogistic Regression:")
accuracy_knn5_lr, f1_knn5_lr = train_model_logistic(X_train_knn, X_test_knn, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['KNN5_XGB'] = accuracy_knn5_xgb
accuracies['KNN5_RF'] = accuracy_knn5_rf
accuracies['KNN5_LR'] = accuracy_knn5_lr
f1_scores['KNN5_XGB'] = f1_knn5_xgb
f1_scores['KNN5_RF'] = f1_knn5_rf
f1_scores['KNN5_LR'] = f1_knn5_lr

In [ ]:
# METHOD 6: KNN Imputation (k=10)
print("=" * 50)
print("METHOD: KNN Imputation (k=10)")
print("=" * 50)

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Apply KNN imputation with k=10
X_train_knn10 = impute_knn(X_train, n_neighbors=10)
X_test_knn10 = impute_knn(X_test, n_neighbors=10)

print("\nXGBoost:")
accuracy_knn10_xgb, f1_knn10_xgb = train_model_XGBoost(X_train_knn10, X_test_knn10, y_train, y_test)
print("\nRandom Forest:")
accuracy_knn10_rf, f1_knn10_rf = train_model_random_forest(X_train_knn10, X_test_knn10, y_train, y_test)
print("\nLogistic Regression:")
accuracy_knn10_lr, f1_knn10_lr = train_model_logistic(X_train_knn10, X_test_knn10, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['KNN10_XGB'] = accuracy_knn10_xgb
accuracies['KNN10_RF'] = accuracy_knn10_rf
accuracies['KNN10_LR'] = accuracy_knn10_lr
f1_scores['KNN10_XGB'] = f1_knn10_xgb
f1_scores['KNN10_RF'] = f1_knn10_rf
f1_scores['KNN10_LR'] = f1_knn10_lr

In [ ]:
# METHOD 7: MissForest Imputation
print("=" * 50)
print("METHOD: MissForest Imputation (Scikit-Learn IterativeImputer)")
print("=" * 50)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Create MissForest imputer using IterativeImputer with RandomForestRegressor
missforest_imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    max_iter=40,
    random_state=42,
    verbose=0
)

# Fit on training data and transform both train and test
missforest_imputer.fit(X_train_original)
X_train_missforest = pd.DataFrame(
    missforest_imputer.transform(X_train_original),
    columns=X_train_original.columns
)
X_test_missforest = pd.DataFrame(
    missforest_imputer.transform(X_test_original),
    columns=X_test_original.columns
)

print("\nXGBoost:")
accuracy_missforest_xgb, f1_missforest_xgb = train_model_XGBoost(X_train_missforest, X_test_missforest, y_train, y_test)
print("\nRandom Forest:")
accuracy_missforest_rf, f1_missforest_rf = train_model_random_forest(X_train_missforest, X_test_missforest, y_train, y_test)
print("\nLogistic Regression:")
accuracy_missforest_lr, f1_missforest_lr = train_model_logistic(X_train_missforest, X_test_missforest, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['MissForest_XGB'] = accuracy_missforest_xgb
accuracies['MissForest_RF'] = accuracy_missforest_rf
accuracies['MissForest_LR'] = accuracy_missforest_lr
f1_scores['MissForest_XGB'] = f1_missforest_xgb
f1_scores['MissForest_RF'] = f1_missforest_rf
f1_scores['MissForest_LR'] = f1_missforest_lr

In [ ]:
# METHOD 8: MICE Imputation
print("=" * 50)
print("METHOD: MICE Imputation (Scikit-Learn IterativeImputer)")
print("=" * 50)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

X_train = X_train_original.copy()
X_test = X_test_original.copy()
y_train = y_train_original.copy()
y_test = y_test_original.copy()

# Create MICE imputer using IterativeImputer with Bayesian Ridge (default for MICE)
mice_imputer = IterativeImputer(
    max_iter=40,
    random_state=42,
    verbose=0
)

# Fit on training data and transform both train and test
mice_imputer.fit(X_train_original)
X_train_mice = pd.DataFrame(
    mice_imputer.transform(X_train_original),
    columns=X_train_original.columns
)
X_test_mice = pd.DataFrame(
    mice_imputer.transform(X_test_original),
    columns=X_test_original.columns
)

print("\nXGBoost:")
accuracy_mice_xgb, f1_mice_xgb = train_model_XGBoost(X_train_mice, X_test_mice, y_train, y_test)
print("\nRandom Forest:")
accuracy_mice_rf, f1_mice_rf = train_model_random_forest(X_train_mice, X_test_mice, y_train, y_test)
print("\nLogistic Regression:")
accuracy_mice_lr, f1_mice_lr = train_model_logistic(X_train_mice, X_test_mice, y_train, y_test)

# Store the accuracies and F1-scores
accuracies['MICE_XGB'] = accuracy_mice_xgb
accuracies['MICE_RF'] = accuracy_mice_rf
accuracies['MICE_LR'] = accuracy_mice_lr
f1_scores['MICE_XGB'] = f1_mice_xgb
f1_scores['MICE_RF'] = f1_mice_rf
f1_scores['MICE_LR'] = f1_mice_lr

In [ ]:
# Translate Dictionary to DataFrame and format results
# Create separate dataframes for accuracy and F1-score

# Ensure accuracies and f1_scores are dictionaries (in case kernel was reset)
if not isinstance(accuracies, dict) or not isinstance(f1_scores, dict):
    print("WARNING: Dictionaries were corrupted. Please re-run cells 10-18 to rebuild results.")
else:
    results_accuracy = []
    results_f1 = []

    # Process accuracies and F1-scores together using the SAME method keys
    for method, accuracy in accuracies.items():
        if '_XGB' in method:
            imputation_method = method.replace('_XGB', '')
            classifier = 'XGBoost'
        elif '_RF' in method:
            imputation_method = method.replace('_RF', '')
            classifier = 'RandomForest'
        elif '_LR' in method:
            imputation_method = method.replace('_LR', '')
            classifier = 'LogisticRegression'
        else:
            imputation_method = method
            classifier = 'XGBoost'
        
        results_accuracy.append({
            'Imputation Method': imputation_method,
            'Classifier': classifier,
            'Accuracy': round(accuracy, 4)
        })
        
        # Get F1-score using the EXACT same key (method) that's in accuracies
        if method in f1_scores:
            f1_value = f1_scores[method]
            results_f1.append({
                'Imputation Method': imputation_method,
                'Classifier': classifier,
                'F1-Score': round(f1_value, 4)
            })

    df_accuracy = pd.DataFrame(results_accuracy)
    df_f1 = pd.DataFrame(results_f1)

    # Print total entries to verify all methods are captured
    print("\nDebug Info:")
    print(f"Total accuracy entries: {len(results_accuracy)}")
    print(f"Total F1-score entries: {len(results_f1)}")
    print(f"Accuracy method keys: {list(accuracies.keys())}")
    print(f"F1-score method keys: {list(f1_scores.keys())}")

    # Pivot to show side-by-side comparison
    if not df_accuracy.empty:
        df_accuracy_pivot = df_accuracy.pivot(index='Imputation Method', columns='Classifier', values='Accuracy')
        print("\n" + "="*70)
        print("RESULTS: Accuracy Comparison (All Methods)")
        print("="*70)
        print(df_accuracy_pivot)

    if not df_f1.empty:
        df_f1_pivot = df_f1.pivot(index='Imputation Method', columns='Classifier', values='F1-Score')
        print("\n" + "="*70)
        print("RESULTS: F1-Score Comparison (All Methods)")
        print("="*70)
        print(df_f1_pivot)

    print("\n" + "="*70)
    print("\nNote: F1-Score is more reliable for imbalanced multi-class data")
    print("Lower values may indicate class imbalance issues.")

# Optimization Notes: Why MICE/MissForest May Not Be Much Better

## Why Complex Methods Don't Always Win

For this dataset, **it's normal** that MICE and MissForest don't dramatically outperform mean/median. Here's why:

### Dataset Characteristics:
- **Small dataset**: 920 samples × 14 features (complex methods need more data)
- **Low missing rate**: ~5-7% missing (imputation quality differences are small)
- **Simple patterns**: Missing values are likely MCAR, not complex MNAR patterns

### Parameters We Optimized:

**MissForest:**
- `n_estimators=200` (up from 100) - More trees = better quality
- `max_iter=40` (up from 30) - More iterations = more refinement

**MICE:**
- `max_iter=40` (up from 30) - More iterations for convergence
- `sample_posterior=False` (changed from True) - Better stability on small datasets
- `n_imputations=5` - Multiple imputations increase robustness

### When Advanced Methods Excel:
Advanced methods like MICE/MissForest typically show ~5-15% improvement over simple methods when:
1. **Missing data is large** (20%+) and **MNAR** (Not Missing At Random)
2. **Dataset is large** (10K+ samples) with **complex relationships**
3. **Missing patterns depend on other variables** (e.g., people with low income less likely to report salary)

### Recommendation:
Given your results, **mean/median imputation is efficient for this dataset**. The computational cost of MICE/MissForest (~10-20x slower) may not justify the gains. However, keep the advanced methods for comparison and documentation purposes.

## Why is accuracy 0.5?

### Findings:
1. **Multi-class classification problem** (not binary!)
   - Target variable has 5 classes: 0, 1, 2, 3, 4
   - Classes are imbalanced:
     - Class 0: 45.7%
     - Class 1: 28.7%
     - Class 2: 11.4%
     - Class 3: 11.0%
     - Class 4: 3.3%

2. **Accuracy of 0.5-0.6 is actually reasonable** for imbalanced multi-class:
   - Random guessing would be 20% (1/5 classes)
   - Predicting majority class: 45.7%
   - **Your model at 59.8% is doing better than baseline!**

3. **F1-Score is more reliable** than accuracy for imbalanced data
   - F1-Score weights precision and recall
   - Better metric for comparing imputation methods

### Why are F1-Scores lower than accuracy?
- Weighted F1 considers class imbalance
- Shows true performance across all classes
- MissForest and MICE may perform worse due to:
  - Over-fitting during iterative imputation
  - Introducing artificial patterns
  - Data distribution shifts


# Part 2: Artificial Missing Data Experiment

Now we'll evaluate imputation quality by:
1. Creating a complete dataset (removing rows with real missing values)
2. Artificially introducing 25% missing values
3. Applying different imputation methods
4. Comparing results using Mean Squared Error (MSE) - lower is better

In [ ]:
# Step 1: Use the SAME complete rows from Part 1 Pairwise Deletion
# This ensures we're comparing apples to apples
print("=" * 70)
print("Using same 308 complete rows as Part 1 Pairwise Deletion")
print("=" * 70)

# Get the complete rows by filtering the original data
df_original_full = read_data()
complete_mask = df_original_full.notna().all(axis=1)
df_complete = df_original_full[complete_mask].copy()

print(f"\nComplete dataset shape: {df_complete.shape}")
print(f"Missing values in complete dataset: {df_complete.isnull().sum().sum()}")

# Now split this complete dataset using the SAME indices as Part 1
# We need to identify which rows from df_complete correspond to the train/test split
# We'll use the same split strategy: split first, then we have the complete subset

# To match Part 1 exactly, we need to:
# 1. Split ALL data first (like Part 1 does)
# 2. Then filter to complete rows only
# 3. This gives us the exact same train/test split as Pairwise Deletion in Part 1

df_temp = read_data()
X_temp = df_temp.drop('num', axis=1)
y_temp = df_temp['num']
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42
)

# Now filter to complete rows only (matching Pairwise deletion logic)
train_mask = X_train_temp.notna().all(axis=1)
X_train_indices = X_train_temp[train_mask].index
y_train_indices = y_train_temp[train_mask].index

test_mask = X_test_temp.notna().all(axis=1)
X_test_indices = X_test_temp[test_mask].index
y_test_indices = y_test_temp[test_mask].index

# Now create df_complete with a marker for train/test split
df_complete['split'] = 'unused'
df_complete.loc[df_complete.index.isin(X_train_indices), 'split'] = 'train'
df_complete.loc[df_complete.index.isin(X_test_indices), 'split'] = 'test'

# Remove the rows that weren't in the original split (shouldn't be any, but just in case)
df_complete = df_complete[df_complete['split'].isin(['train', 'test'])].copy()

print(f"Train rows: {(df_complete['split'] == 'train').sum()}")
print(f"Test rows: {(df_complete['split'] == 'test').sum()}")

In [ ]:
# Step 2: Artificially introduce 25% missing values in numerical columns
# Using the same complete data with the same train/test split
np.random.seed(42)
missing_fraction = 0.25

# Create copy for artificial missingness (excluding the 'split' column)
df_artificial = df_complete.copy()

# Store original values and positions where we'll introduce missing data
original_values = {}
missing_positions = {}

# Get numerical columns (exclude target variable 'num' and 'split' marker)
numerical_cols = df_artificial.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['num', 'split']]

print(f"\nIntroducing {missing_fraction*100}% missing values in {len(numerical_cols)} numerical columns...")
print(f"Numerical columns: {numerical_cols}\n")

for col in numerical_cols:
    n_missing = int(len(df_artificial) * missing_fraction)
    missing_idx = np.random.choice(df_artificial.index, size=n_missing, replace=False)
    
    # Store original values before introducing missing data
    original_values[col] = df_artificial.loc[missing_idx, col].copy()
    missing_positions[col] = missing_idx
    
    # Introduce missing values
    df_artificial.loc[missing_idx, col] = np.nan

print(f"Total missing values introduced: {df_artificial.isnull().sum().sum()}")
print(f"Percentage of dataset now missing: {(df_artificial.isnull().sum().sum() / (df_artificial.shape[0] * df_artificial.shape[1]) * 100):.2f}%")

In [ ]:
# Step 3: Define function to calculate MSE for imputation quality
def calculate_mse(original_values_dict, imputed_df, missing_positions_dict):
    """
    Calculate Mean Squared Error between original values and imputed values.
    
    Parameters:
    - original_values_dict: dict with column -> original values (Series)
    - imputed_df: DataFrame with imputed values
    - missing_positions_dict: dict with column -> indices where values were removed
    
    Returns:
    - mse: Mean Squared Error across all imputed values
    """
    all_original = []
    all_imputed = []
    
    for col in original_values_dict.keys():
        orig_vals = original_values_dict[col].values
        imputed_vals = imputed_df.loc[missing_positions_dict[col], col].values
        
        all_original.extend(orig_vals)
        all_imputed.extend(imputed_vals)
    
    mse = mean_squared_error(all_original, all_imputed)
    return mse

print("MSE calculation function defined.")

In [ ]:
# Step 4: Apply imputation methods and calculate MSE for each
mse_results = {}

print("=" * 70)
print("EVALUATING IMPUTATION QUALITY WITH MSE (Lower is Better)")
print("=" * 70)

# Method 1: Mean Imputation
print("\n1. Mean Imputation")
df_mean = df_artificial.copy()
imputer_mean = SimpleImputer(strategy='mean')
df_mean[numerical_cols] = imputer_mean.fit_transform(df_mean[numerical_cols])
mse_mean = calculate_mse(original_values, df_mean, missing_positions)
mse_results['Mean'] = mse_mean
print(f"   MSE: {mse_mean:.4f}")

# Method 2: Median Imputation
print("\n2. Median Imputation")
df_median = df_artificial.copy()
imputer_median = SimpleImputer(strategy='median')
df_median[numerical_cols] = imputer_median.fit_transform(df_median[numerical_cols])
mse_median = calculate_mse(original_values, df_median, missing_positions)
mse_results['Median'] = mse_median
print(f"   MSE: {mse_median:.4f}")

# Method 3: KNN Imputation (k=5)
print("\n3. KNN Imputation (k=5)")
df_knn5 = df_artificial.copy()
imputer_knn5 = KNNImputer(n_neighbors=5)
df_knn5[numerical_cols] = imputer_knn5.fit_transform(df_knn5[numerical_cols])
mse_knn5 = calculate_mse(original_values, df_knn5, missing_positions)
mse_results['KNN (k=5)'] = mse_knn5
print(f"   MSE: {mse_knn5:.4f}")

# Method 4: KNN Imputation (k=10)
print("\n4. KNN Imputation (k=10)")
df_knn10 = df_artificial.copy()
imputer_knn10 = KNNImputer(n_neighbors=10)
df_knn10[numerical_cols] = imputer_knn10.fit_transform(df_knn10[numerical_cols])
mse_knn10 = calculate_mse(original_values, df_knn10, missing_positions)
mse_results['KNN (k=10)'] = mse_knn10
print(f"   MSE: {mse_knn10:.4f}")

print("\n" + "=" * 70)

In [ ]:

# Method 5: MissForest Imputation
print("\n5. MissForest Imputation")
df_missforest_eval = df_artificial.copy()
df_missforest_eval = impute_missForest(df_missforest_eval)
mse_missforest = calculate_mse(original_values, df_missforest_eval, missing_positions)
mse_results['MissForest'] = mse_missforest
print(f"   MSE: {mse_missforest:.4f}")

# Method 6: MICE Imputation
print("\n6. MICE Imputation (5 imputations)")
df_artificial_mice = df_artificial.copy()
mice_imputations_eval = impute_mice(df_artificial_mice, n_imputations=5, max_iter=40)
df_mice_eval = pool_mice_results(mice_imputations_eval)
mse_mice = calculate_mse(original_values, df_mice_eval, missing_positions)
mse_results['MICE'] = mse_mice
print(f"   MSE: {mse_mice:.4f}")

print("\n" + "=" * 70)


In [ ]:
# Step 5: Display MSE comparison results
mse_df = pd.DataFrame(list(mse_results.items()), columns=['Imputation Method', 'MSE'])
mse_df = mse_df.sort_values('MSE')
mse_df['MSE'] = mse_df['MSE'].round(4)

print("\n" + "=" * 60)
print("MSE COMPARISON RESULTS (Lower is Better)")
print("=" * 60)
mse_df

## Now let's compare prediction accuracy

We'll train RandomForest on:
1. The real complete dataset (baseline)
2. Each imputed dataset

This will show if lower MSE (better imputation) leads to higher prediction accuracy.

In [ ]:
# Use the SAME train/test split from Part 1 (already done on the full 920 rows)
# Filter to get only the complete rows from that split
from sklearn.model_selection import train_test_split

# Dictionary to store accuracy results
accuracy_results = {}

print("=" * 70)
print("TRAINING RANDOM FOREST ON DIFFERENT DATASETS")
print("=" * 70)

# 1. Baseline: Train on real complete data using the SAME split as Part 1
print("\n1. Baseline - Real Complete Data (No Missing Values)")

# Use the original split indices, but filter to only complete rows
# This matches what Pairwise Deletion does in Part 1
X_train_baseline = X_train_original.copy()
X_test_baseline = X_test_original.copy()
y_train_baseline = y_train_original.copy()
y_test_baseline = y_test_original.copy()

# Filter to complete rows only (same as Pairwise Deletion)
train_mask = X_train_baseline.notna().all(axis=1)
X_train_baseline = X_train_baseline[train_mask]
y_train_baseline = y_train_baseline[train_mask]

test_mask = X_test_baseline.notna().all(axis=1)
X_test_baseline = X_test_baseline[test_mask]
y_test_baseline = y_test_baseline[test_mask]

print(f"   Train size: {X_train_baseline.shape[0]}, Test size: {X_test_baseline.shape[0]}")

rf_baseline = RandomForestClassifier(random_state=42)
rf_baseline.fit(X_train_baseline, y_train_baseline)
y_pred_baseline = rf_baseline.predict(X_test_baseline)
acc_baseline = accuracy_score(y_test_baseline, y_pred_baseline)
accuracy_results['Baseline (Real Data)'] = acc_baseline
print(f"   Accuracy: {acc_baseline:.4f}")

print("\n" + "=" * 70)

In [ ]:
# For imputed datasets, we need to:
# 1. Map the 308 complete row indices back to train/test split
# 2. Use the SAME train/test division as Part 1

# Get the indices of complete rows in original data
complete_indices = df_complete.index

# Find which complete rows were in train vs test in the original split
train_complete_indices = [idx for idx in X_train_original.index if idx in complete_indices]
test_complete_indices = [idx for idx in X_test_original.index if idx in complete_indices]

print(f"\nUsing original split: {len(train_complete_indices)} train + {len(test_complete_indices)} test = {len(complete_indices)} total complete rows")

# 2. Train on Mean Imputed data
print("\n2. Mean Imputation")
X_train_mean = df_mean.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_mean = df_mean['num'].loc[train_complete_indices]
X_test_mean = df_mean.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_mean = df_mean['num'].loc[test_complete_indices]

rf_mean = RandomForestClassifier(random_state=42)
rf_mean.fit(X_train_mean, y_train_mean)
y_pred_mean = rf_mean.predict(X_test_mean)
acc_mean = accuracy_score(y_test_mean, y_pred_mean)
accuracy_results['Mean'] = acc_mean
print(f"   MSE: {mse_results['Mean']:.4f}")
print(f"   Accuracy: {acc_mean:.4f}")

# 3. Train on Median Imputed data
print("\n3. Median Imputation")
X_train_median = df_median.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_median = df_median['num'].loc[train_complete_indices]
X_test_median = df_median.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_median = df_median['num'].loc[test_complete_indices]

rf_median = RandomForestClassifier(random_state=42)
rf_median.fit(X_train_median, y_train_median)
y_pred_median = rf_median.predict(X_test_median)
acc_median = accuracy_score(y_test_median, y_pred_median)
accuracy_results['Median'] = acc_median
print(f"   MSE: {mse_results['Median']:.4f}")
print(f"   Accuracy: {acc_median:.4f}")

# 4. Train on KNN (k=5) Imputed data
print("\n4. KNN Imputation (k=5)")
X_train_knn5 = df_knn5.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_knn5 = df_knn5['num'].loc[train_complete_indices]
X_test_knn5 = df_knn5.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_knn5 = df_knn5['num'].loc[test_complete_indices]

rf_knn5 = RandomForestClassifier(random_state=42)
rf_knn5.fit(X_train_knn5, y_train_knn5)
y_pred_knn5 = rf_knn5.predict(X_test_knn5)
acc_knn5 = accuracy_score(y_test_knn5, y_pred_knn5)
accuracy_results['KNN (k=5)'] = acc_knn5
print(f"   MSE: {mse_results['KNN (k=5)']:.4f}")
print(f"   Accuracy: {acc_knn5:.4f}")

# 5. Train on KNN (k=10) Imputed data
print("\n5. KNN Imputation (k=10)")
X_train_knn10 = df_knn10.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_knn10 = df_knn10['num'].loc[train_complete_indices]
X_test_knn10 = df_knn10.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_knn10 = df_knn10['num'].loc[test_complete_indices]

rf_knn10 = RandomForestClassifier(random_state=42)
rf_knn10.fit(X_train_knn10, y_train_knn10)
y_pred_knn10 = rf_knn10.predict(X_test_knn10)
acc_knn10 = accuracy_score(y_test_knn10, y_pred_knn10)
accuracy_results['KNN (k=10)'] = acc_knn10
print(f"   MSE: {mse_results['KNN (k=10)']:.4f}")
print(f"   Accuracy: {acc_knn10:.4f}")

print("\n" + "=" * 70)

In [ ]:

# 6. Train on MissForest Imputed data
print("\n6. MissForest Imputation")
X_train_missforest_eval = df_missforest_eval.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_missforest_eval = df_missforest_eval['num'].loc[train_complete_indices]
X_test_missforest_eval = df_missforest_eval.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_missforest_eval = df_missforest_eval['num'].loc[test_complete_indices]

rf_missforest = RandomForestClassifier(random_state=42)
rf_missforest.fit(X_train_missforest_eval, y_train_missforest_eval)
y_pred_missforest = rf_missforest.predict(X_test_missforest_eval)
acc_missforest = accuracy_score(y_test_missforest_eval, y_pred_missforest)
accuracy_results['MissForest'] = acc_missforest
print(f"   MSE: {mse_results['MissForest']:.4f}")
print(f"   Accuracy: {acc_missforest:.4f}")

# 7. Train on MICE Imputed data
print("\n7. MICE Imputation (5 imputations)")
X_train_mice_eval = df_mice_eval.drop(['num', 'split'], axis=1).loc[train_complete_indices]
y_train_mice_eval = df_mice_eval['num'].loc[train_complete_indices]
X_test_mice_eval = df_mice_eval.drop(['num', 'split'], axis=1).loc[test_complete_indices]
y_test_mice_eval = df_mice_eval['num'].loc[test_complete_indices]

rf_mice = RandomForestClassifier(random_state=42)
rf_mice.fit(X_train_mice_eval, y_train_mice_eval)
y_pred_mice = rf_mice.predict(X_test_mice_eval)
acc_mice = accuracy_score(y_test_mice_eval, y_pred_mice)
accuracy_results['MICE'] = acc_mice
print(f"   MSE: {mse_results['MICE']:.4f}")
print(f"   Accuracy: {acc_mice:.4f}")

print("\n" + "=" * 70)


In [ ]:
# Create comprehensive comparison table
comparison_data = []

# Add baseline (no MSE since it's real data)
comparison_data.append({
    'Method': 'Baseline (Real Data)',
    'MSE': 'N/A',
    'Accuracy': accuracy_results['Baseline (Real Data)']
})

# Add imputation methods
for method in ['Mean', 'Median', 'KNN (k=5)', 'KNN (k=10)', 'MissForest', 'MICE']:
    comparison_data.append({
        'Method': method,
        'MSE': mse_results[method],
        'Accuracy': accuracy_results[method]
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df['Accuracy'] = comparison_df['Accuracy'].apply(lambda x: round(x, 4))
comparison_df['MSE'] = comparison_df['MSE'].apply(lambda x: round(x, 4) if isinstance(x, (int, float)) else x)

# Sort by accuracy (descending)
comparison_df_sorted = comparison_df.copy()
comparison_df_sorted['Accuracy_num'] = comparison_df_sorted['Accuracy']
comparison_df_sorted = comparison_df_sorted.sort_values('Accuracy_num', ascending=False)
comparison_df_sorted = comparison_df_sorted.drop('Accuracy_num', axis=1)

print("\n" + "=" * 70)
print("FINAL COMPARISON: MSE vs Prediction Accuracy")
print("=" * 70)
print("\nSorted by Accuracy (Higher is Better):")
comparison_df_sorted
